In [1]:
import matplotlib.pyplot as plt
import numpy as np

from snn_dpe.tools.network import create_encoders, create_network, run_network_timeseries
from snn_dpe.tools.data import Mackey_Glass, util

from snn_dpe.tools.train import forward_pass, mse, update_weights, train_TS, test_timeseries
from tqdm import tqdm
from snn_dpe.tools.plotting import *

In [3]:
MG_data_tr = Mackey_Glass.read_MG_data('./Data/MarkusThill-MGAB-60f6b17/1.csv', normalize=True)
MG_data_te = Mackey_Glass.read_MG_data('./Data/MarkusThill-MGAB-60f6b17/2.csv', normalize=True)

# slice up MG into input, output pairs
input_window_size = 10
output_window_size = 1


n_tr_data = 2000
n_te_data = 2000

MG_inputs_tr, MG_outputs_tr = util.generate_io_pairs(MG_data_tr[:n_tr_data], input_window_size, output_window_size)
MG_inputs_te, MG_outputs_te = util.generate_io_pairs(MG_data_te[:n_te_data], input_window_size, output_window_size)


# create a test network
n_neurons = 30
synapse_density = 0.1
n_input = 12 #how many neurons will receive potentiation adjustments related to MG
n_synapses = int((n_neurons**2)*synapse_density)

neurons = create_network(n_neurons, n_synapses, negative_weights = True, threshold_range = (0.35, 1), leak_range = (0.05, 0.25), weight_factor = 1)

n_epochs = 3

noise_axis = np.linspace(0, 0.20, num=20)
tr_mses = []
te_mses = []

for n in noise_axis:
    tr_mses.append([])
    te_mses.append([])

for _ in tqdm(range(10)):
    for i, n in enumerate(noise_axis):
        training_mses, testing_mses, relative_dpe_weights, dpe_bias_relative = train_TS(n_epochs, MG_inputs_tr, MG_outputs_tr, neurons, n_input, TS_inputs_te=MG_inputs_te, TS_outputs_te=MG_outputs_te, relative=True, write_noise_std=n)
        tr_mses[i].append(training_mses[-1])
        te_mses[i].append(testing_mses[-1])




 60%|██████    | 6/10 [11:55<08:20, 125.01s/it]/home/rfebbo/git/SNN_DPE/src/snn_dpe/tools/train.py:123: RuntimeWarning: overflow encountered in add
  dpe_weights += np.random.choice([-1,1], dpe_weights.shape)*dpe_weights*write_noise_std
/home/rfebbo/git/SNN_DPE/src/snn_dpe/tools/train.py:31: RuntimeWarning: invalid value encountered in double_scalars
  dpe_weights[i][j] -= e[j] * x[i] * lr
/home/rfebbo/git/SNN_DPE/src/snn_dpe/tools/train.py:31: RuntimeWarning: invalid value encountered in subtract
  dpe_weights[i][j] -= e[j] * x[i] * lr
/home/rfebbo/git/SNN_DPE/src/snn_dpe/tools/train.py:123: RuntimeWarning: invalid value encountered in add
  dpe_weights += np.random.choice([-1,1], dpe_weights.shape)*dpe_weights*write_noise_std
100%|██████████| 10/10 [19:11<00:00, 115.14s/it]


In [ ]:
plt.plot(noise_axis, tr_mses, label='Training')
plt.plot(noise_axis, te_mses, label='Testing')
plt.legend()
plt.xlabel('Noise std. dev.')
plt.ylabel('Mean Squared Error')
plt.show()